Installing Necessary Libraries

In [ ]:
!pip install gradio transformers diffusers accelerate xformers

Importing Libraries


In [ ]:
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import torch
import gradio as gr

Load the Stable Diffusion Model

In [ ]:
def load_model(model_id):
    pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
    pipe = pipe.to("cuda")
    return pipe

Load the Default Model

In [ ]:
default_model_id = "stabilityai/stable-diffusion-2-1-base"  # A model supporting high-resolution generation
pipe = load_model(default_model_id)

Function to Generate High-Resolution Images

In [ ]:
def generate_high_res_image(prompt, model_id, resolution):
    global pipe

    # Check if the model ID has changed
    if model_id != default_model_id:
        pipe = load_model(model_id)

    # Generate the image in base resolution (e.g., 512x512)
    image = pipe(prompt, guidance_scale=7.5).images[0]

    # Upscale the image to desired resolution using bicubic interpolation (fallback)
    upscale_factor = resolution // 512
    high_res_image = image.resize((512 * upscale_factor, 512 * upscale_factor), Image.BICUBIC)
    return high_res_image

Gradio Interface

In [ ]:
def main_interface():
    interface = gr.Interface(
        fn=generate_high_res_image,
        inputs=[
            gr.Textbox(
                lines=2,
                placeholder="Enter your prompt here (e.g., A fashion model walking on a ramp).",
                label="Text Prompt",
            ),
            gr.Textbox(
                value=default_model_id,
                label="Model ID",
                placeholder="Enter the model ID (e.g., stabilityai/stable-diffusion-2-1).",
            ),
            gr.Slider(
                minimum=512,
                maximum=2048,
                step=512,
                value=2048,
                label="Output Resolution (Width = Height)",
            ),
        ],
        outputs=gr.Image(type="pil", label="Generated High-Resolution Image"),
        title="High-Resolution Text-to-Image Generator",
        description=(
            "This application generates photorealistic images based on text prompts using Stable Diffusion. "
            "Enter a text description and choose a model to generate an image. The default model is "
            "'stabilityai/stable-diffusion-2-1-base', and images can be generated at resolutions up to 2048x2048."
        ),
        allow_flagging="never",
    )
    return interface

Launch the Gradio App

In [ ]:
interface = main_interface()
interface.launch(debug=True)